# Using Upstage Document Parse on SageMaker Jumpstart

Upstage Documnet Parse is a powerful API designed to automatically convert any document to HTML. It detects layout elements such as paragraphs, tables, images, and more to determine the structure of the document. The API then serializes the elements according to reading order, and finally converts the document into HTML.

This sample notebook shows you how to deploy [Upstage Document Parse](https://aws.amazon.com/marketplace/pp/prodview-lv5bnpdco7xoq) using Amazon SageMaker.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
2. Ensure that IAM role used has **AmazonSageMakerFullAccess**
3. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        2. **aws-marketplace:Unsubscribe**
        3. **aws-marketplace:Subscribe**  

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Prepare input payload](#B.-Prepare-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
3. [Clean-up](#4.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   2. [Delete the model](#B.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open [Upstage Document Parse](https://aws.amazon.com/marketplace/pp/prodview-lv5bnpdco7xoq) model package listing page.
2. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
3. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
4. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [9]:
%pip install -q sagemaker==2.253.1 boto3==1.40.59 requests_toolbelt

Note: you may need to restart the kernel to use updated packages.


In [1]:
import boto3
import sagemaker
from sagemaker import ModelPackage, get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml


In [ ]:
#role = get_execution_role()
role = "arn:aws:iam::6**2*****98*:role/service-role/AmazonSageMaker-ExecutionRole-20241***T132***" # <your role arn>
sagemaker_session = sagemaker.Session()
sagemaker_runtime = boto3.client("sagemaker-runtime")

In [4]:
model_package_name = "upstage-document-parse-250404--d656919d867339d6ab3a2b645182a28b"
# if you want to use ml.g5 instances, use below package
# model_package_name = "upstage-document-parse-250404--b6bdea6dc54636a9b2578bd32ac7393f"

# Mapping for Model Packages
model_package_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:model-package/{model_package_name}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:model-package/{model_package_name}",
    "us-west-1": f"arn:aws:sagemaker:us-west-1:382657785993:model-package/{model_package_name}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:model-package/{model_package_name}",
    "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:model-package/{model_package_name}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:model-package/{model_package_name}",
    "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:model-package/{model_package_name}",
    "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:model-package/{model_package_name}",
    "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:model-package/{model_package_name}",
    "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:model-package/{model_package_name}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/{model_package_name}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/{model_package_name}",
    "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/{model_package_name}",
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/{model_package_name}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:model-package/{model_package_name}",
    "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:model-package/{model_package_name}",
}

region = sagemaker_session.boto_region_name
if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = model_package_map[region]

print(f"Model Package: '{model_package_arn}'")

region = sagemaker_session.boto_region_name
if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = (
    model_package_map[region]
)

print(f"Model Package: '{model_package_arn}'")

Model Package: 'arn:aws:sagemaker:us-west-2:594846645681:model-package/upstage-document-parse-250404--d656919d867339d6ab3a2b645182a28b'
Model Package: 'arn:aws:sagemaker:us-west-2:594846645681:model-package/upstage-document-parse-250404--d656919d867339d6ab3a2b645182a28b'


## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [5]:
model_name = "Upstage-Document-Parse-Test"

real_time_inference_instance_type = (
    "ml.g6.xlarge"
    # if you want to use ml.g5 instances, use `upstage-document-parse-cuda11--a2f49bd4aea13c7b94ba3c034aa524a7` package.
    # "ml.g5.xlarge"
)

### A. Create an endpoint

In [6]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

endpoint_name = sagemaker.utils.name_from_base(model_name)
print(f"endpoint name: '{endpoint_name}'")

endpoint name: 'Upstage-Document-Parse-Test-2025-10-25-08-18-31-168'


In [7]:
# Deploy the model
model.deploy(1, real_time_inference_instance_type, endpoint_name=endpoint_name)

---------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Prepare input payload

We support JPEG, PNG, BMP, PDF, TIFF, HEIC, DOCX, PPTX, XLSX formats.

### C. Perform real-time inference

In [10]:
import boto3
import json
from requests_toolbelt import MultipartEncoder

runtime_sm_client = boto3.client('runtime.sagemaker')

# Prepare multipart form data
m = MultipartEncoder(
    fields={
        'document': ('sample.pdf', open('../data/sample.pdf', 'rb'), 'application/pdf'),
        # 'document': ('solar.pdf', open('solar.pdf', 'rb'), 'application/pdf'), # for PDF files
        'model': 'document-parse',
        "ocr": 'auto', # force
        'coordinates': 'true',
        'output_formats': '["text", "html", "markdown"]',
        'base64_encoding': '["table"]',
        'chart_recognition': 'true',
    }
)

# Get the raw bytes of the multipart form data
body = m.to_string()

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=m.content_type,  # This will be 'multipart/form-data; boundary=...'
    Body=body
)

result = response["Body"].read()

# Print the result
print('output:', json.loads(result))

output: {'api': '2.0', 'content': {'html': '<p id=\'0\' data-category=\'paragraph\' style=\'font-size:14px\'>핵 심 설 명 서</p>\n<br><p id=\'1\' data-category=\'paragraph\' style=\'font-size:18px\'>이 설명서는 금융소비자의 권익 보호 및 금융상품에 대한 이해 증진을 위하여<br>금융상품의 핵심내용을 쉽게 이해하실 수 있도록 작성한 것입니다. 상품내용을<br>충분히 이해하신 다음에 청약여부를 결정하시기 바랍니다.</p>\n<h1 id=\'2\' style=\'font-size:22px\'>핵심설명서</h1>\n<h1 id=\'3\' style=\'font-size:20px\'>[미래에셋증권 제 253 회 파생결합증권(DLS)(원금보장형)]</h1>\n<h1 id=\'4\' style=\'font-size:18px\'>1</h1>\n<br><h1 id=\'5\' style=\'font-size:18px\'>상품 개요</h1>\n<p id=\'6\' data-category=\'list\' style=\'font-size:16px\'>○ 모집가액: 100 억원 (모집된 금액이 10 억 미만일 경우 발행이 취소될 수 있음)<br>○ 기초자산: KOSPI200 지수(블룸버그 Ticker: KOSPI2 index),<br>금가격지수 (블룸버그 Ticker: GOLDLNPM index)<br>○ 청약기간 : 2011 년 07 월 04 일 ~2011 년 07 월 07 일 16 시<br>○ 청약단위 : 100 만원 이상 100 만원 단위 (발행 후 추가 납입 불가)<br>○ 발 행 일 : 2011 년 07 월 08 일<br>○ 만 기 일 : 2012 년 07 월 09 일(만기 1 년)<br>○ 상품위험등급: 4 등급</p>\n<h1 id=\'7\' style=\'font-size:18px\'>2</h1>\n<br><h1 id=\'8

In [11]:
res = json.loads(result)

In [12]:
res.keys()

dict_keys(['api', 'content', 'elements', 'merged_elements', 'model', 'ocr', 'usage'])

In [13]:
res["content"].keys()

dict_keys(['html', 'markdown', 'text'])

In [14]:
res["elements"]#.keys()

[{'category': 'paragraph',
  'content': {'html': "<p id='0' data-category='paragraph' style='font-size:14px'>핵 심 설 명 서</p>",
   'markdown': '핵 심 설 명 서',
   'text': '핵 심 설 명 서'},
  'coordinates': [{'x': 0.0817, 'y': 0.0209},
   {'x': 0.1925, 'y': 0.0209},
   {'x': 0.1925, 'y': 0.0653},
   {'x': 0.0817, 'y': 0.0653}],
  'id': 0,
  'page': 1},
 {'category': 'paragraph',
  'content': {'html': "<br><p id='1' data-category='paragraph' style='font-size:18px'>이 설명서는 금융소비자의 권익 보호 및 금융상품에 대한 이해 증진을 위하여<br>금융상품의 핵심내용을 쉽게 이해하실 수 있도록 작성한 것입니다. 상품내용을<br>충분히 이해하신 다음에 청약여부를 결정하시기 바랍니다.</p>",
   'markdown': '이 설명서는 금융소비자의 권익 보호 및 금융상품에 대한 이해 증진을 위하여\n금융상품의 핵심내용을 쉽게 이해하실 수 있도록 작성한 것입니다. 상품내용을\n충분히 이해하신 다음에 청약여부를 결정하시기 바랍니다.',
   'text': '이 설명서는 금융소비자의 권익 보호 및 금융상품에 대한 이해 증진을 위하여\n금융상품의 핵심내용을 쉽게 이해하실 수 있도록 작성한 것입니다. 상품내용을\n충분히 이해하신 다음에 청약여부를 결정하시기 바랍니다.'},
  'coordinates': [{'x': 0.2009, 'y': 0.0173},
   {'x': 0.9314, 'y': 0.0173},
   {'x': 0.9314, 'y': 0.0702},
   {'x': 0.2009, 'y': 0.0702}],
  'id': 1,

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you can delete the endpoint and avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [ ]:
model.delete_model()